<span style="font-family: Open Sans; font-weight: 800; font-size: 40px; font-style: normal; text-align:center;">

Especialización Inteligencia Artificial - CEIA

</span>

<span style="font-family: Open Sans; font-weight: 700; font-size: 30px; font-style: normal; text-align:center;">

Visión por Computadora I

Trabajo práctico Nro. III

</span>

<span style="font-family: Open Sans; font-weight: 400; font-size: 20px; font-style: normal;">




</span>

# Tabla de Contenido

[Consigna](#Consigna)<br>
[Punto I](#Parte-I)<br>
[Punto II](#Parte-II)<br>
[Punto III](#Parte-II)<br>

# Consigna

▪ Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template.

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.

[inicio](#Tabla-de-contenido)

# Punto 0

[inicio](#Tabla-de-contenido)

In [1]:
# Importamos librerías
%matplotlib inline  
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import argparse
import imutils
import glob
from skimage import io, color, transform, feature
from sklearn import svm

In [2]:
# Cargamos el Template
template = cv.imread('material_TP3/template/pattern.PNG', cv.IMREAD_GRAYSCALE)

# Cargamos las imágenes
img1 = cv.imread('material_TP3/images/coca_logo_1.PNG')
img2 = cv.imread('material_TP3/images/coca_logo_2.PNG')
img3 = cv.imread('material_TP3/images/coca_multi.PNG')
img4 = cv.imread('material_TP3/images/coca_retro_1.PNG')
img5 = cv.imread('material_TP3/images/coca_retro_2.PNG')
img6 = cv.imread('material_TP3/images/COCA-COLA-LOGO.jpg')
img7 = cv.imread('material_TP3/images/logo_1.PNG')


# Las mostramos para chequear que se cargaron bien
cv.imshow('Imagen Original 1 - Template', template)

cv.imshow('Imagen Original 1 - coca_logo_1', img1)
cv.imshow('Imagen Original 2 - coca_logo_2', img2)
cv.imshow('Imagen Original 3 - coca_multi', img3)
cv.imshow('Imagen Original 4 - coca_retro_1', img4)
cv.imshow('Imagen Original 5 - coca_retro_2', img5)
cv.imshow('Imagen Original 6 - COCA-COLA-LOGO', img6)
cv.imshow('Imagen Original 7 - logo_1', img7)

cv.waitKey(0)
cv.destroyAllWindows()

# Punto 1

[inicio](#Tabla-de-contenido)

▪ Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template.

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos

Aplicamos el Detector de Bordes Canny

Búsqueda de un template en varias escalas de una imagen

# Punto 2

[inicio](#Tabla-de-contenido)

2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1

Para cumplir con este punto vamos a utilizar el métomo de "TM_CCOEFF_NORMED" para lograr el matcheo. Previamente redimensionar el template para conseguir el mayor nro. de coincidencias y hacer pruebas con diferentes valores de threshold. Se utilizó el filtro de contorno de canny para conseguir mayor exactitud y eliminar algunos falsos positivos. Además, luego se agregó la supresión de no máximos para evitar solapamiento como vimos en la clase.

In [3]:
# Transformamos la imagen de "coca_multi" a escala de grises
img_gray = cv.cvtColor(img3, cv.COLOR_BGR2GRAY)

# Invertimos los colores del template
template = cv.bitwise_not(template)

# Redimensionamos el template para conseguir la mayor cantidad de coincidencias
nuevo_ancho = int(template.shape[1] * 0.19)
nuevo_alto = int(template.shape[0] * 0.25)
template_redimensionado = cv.resize(template, (nuevo_ancho, nuevo_alto))

w, h = template_redimensionado.shape[::-1]
res = cv.matchTemplate(img_gray, template_redimensionado, cv.TM_CCOEFF_NORMED)
threshold = 0.47

loc = np.where(res >= threshold)

# Aplicamos supresión de no máximos
detections = []
for pt in zip(*loc[::-1]):
    x, y = pt
    if not any(x - w <= xx <= x + w and y - h <= yy <= y + h for xx, yy in detections):
        
        # Aplicamos filtro de contorno de Canny a la región de detección
        roi = img_gray[y:y+h, x:x+w]
        edges = cv.Canny(roi, threshold1=140, threshold2=150)
        
        if np.sum(edges) > 0:
            detections.append((x, y))

# Dibujamos el rectángulo en las detecciones después de la supresión de no máximos y el filtro de contorno
for pt in detections:
    cv.rectangle(img3, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)

cv.imshow("Detecciones", img3)
cv.waitKey(0)
cv.destroyAllWindows()


# Punto 3

[inicio](#Tabla-de-contenido)

3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.

In [4]:
def detect_and_visualize(image_path, template_path, template_width, template_height, threshold):
    # Cargamos la imagen y el template
    img_rgb = cv.imread(image_path)
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
    
    # Invertimos colores del template
    template = cv.bitwise_not(template)
    
    # Redimensionamos el template
    template_resized = cv.resize(template, (template_width, template_height))
    
    w, h = template_resized.shape[::-1]
    res = cv.matchTemplate(img_gray, template_resized, cv.TM_CCOEFF_NORMED)
    
    loc = np.where(res >= threshold)
    
    # Aplicamos supresión de no máximos
    detections = []
    for pt in zip(*loc[::-1]):
        x, y = pt
        if not any(x - w <= xx <= x + w and y - h <= yy <= y + h for xx, yy in detections):
            
            # Aplicamos filtro de contorno de Canny a la región de detección
            roi = img_gray[y:y+h, x:x+w]
            edges = cv.Canny(roi, threshold1=140, threshold2=150)
            
            if np.sum(edges) > 0:
                detections.append((x, y))
    
    # Dibujamos el rectángulo en las detecciones después de la supresión de no máximos y el filtro de contorno
    for pt in detections:
        cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
    
    # Mostramos imagen con detecciones marcadas y nivel de confianza
    for pt in detections:
        confidence = res[pt[1], pt[0]]
        cv.putText(img_rgb, f'Conf: {confidence:.2f}', (pt[0], pt[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    cv.imshow("Detecciones", img_rgb)
    cv.waitKey(0)
    cv.destroyAllWindows()



In [5]:
# Llamamos a la función seteando los parámetros en los argumentos

detect_and_visualize('material_TP3/images/coca_logo_1.PNG', 'material_TP3/template/pattern.PNG', 135, 60, threshold=0.2)
detect_and_visualize('material_TP3/images/coca_logo_2.png', 'material_TP3/template/pattern.PNG', 180, 90, threshold=0.195)
detect_and_visualize('material_TP3/images/COCA-COLA-LOGO.jpg', 'material_TP3/template/pattern.PNG', 1200, 400, threshold=0.195)
detect_and_visualize('material_TP3/images/coca_multi.png', 'material_TP3/template/pattern.PNG', 90, 40, threshold=0.47)
detect_and_visualize('material_TP3/images/coca_retro_1.png', 'material_TP3/template/pattern.PNG', 470, 150, threshold=0.25)
detect_and_visualize('material_TP3/images/coca_retro_2.png', 'material_TP3/template/pattern.PNG', 150, 50, threshold=0.37)
detect_and_visualize('material_TP3/images/logo_1.png', 'material_TP3/template/pattern.PNG', 260, 100, threshold=0.237)